# 0. 라이브러리 호출 및 데이터, 모델 로드

## 라이브러리 호출

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Colab Notebooks/여친_TravelMate

/content/drive/MyDrive/Colab Notebooks/여친_TravelMate


In [3]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 8.7 MB/s eta 0:00:00


In [4]:
import pandas as pd
from catboost import CatBoostRegressor

## 데이터 로드

In [5]:
df = pd.read_csv('./Data/data_v5.csv')
df

,VISIT_AREA_ID,TRAVEL_ID,TRAVELER_ID,VISIT_AREA_NM,TRAVEL_NM,VISIT_AREA_TYPE_CD,DGSTFN,REVISIT_INTENTION,RCMDTN_INTENTION,RES_CAFE,...,TRAVEL_STYL_3,TRAVEL_STYL_4,TRAVEL_STYL_5,TRAVEL_STYL_6,TRAVEL_STYL_7,TRAVEL_STYL_8,TRAVEL_MOTIVE_1,TRAVEL_STATUS_ACCOMPANY,TRAVEL_COMPANIONS_NUM,SAT
0,2208190004,d_d000296,d000296,산굼부리,D01,1,5.0,5.0,5.0,2,...,3,2,5,4,2,5,3,자녀 동반 여행,3,2
1,2208180010,d_d000296,d000296,한림리조트,D01,24,5.0,5.0,5.0,2,...,3,2,5,4,2,5,3,자녀 동반 여행,3,2
2,2208200001,d_d000296,d000296,스누피가든,D01,6,5.0,5.0,5.0,2,...,3,2,5,4,2,5,3,자녀 동반 여행,3,2
3,2208180007,d_d000296,d000296,협재해수욕장,D01,1,5.0,5.0,5.0,2,...,3,2,5,4,2,5,3,자녀 동반 여행,3,2
4,2208180004,d_d000296,d000296,카페노티드 제주애월,D01,11,1.0,5.0,5.0,1,...,3,2,5,4,2,5,3,자녀 동반 여행,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31772,2208210008,d_d000621,d000621,장동민의크라우드pc방,D01,3,5.0,5.0,5.0,2,...,2,2,4,2,3,5,1,2인 가족 여행,1,2
31773,2208210008,d_d000621,d000621,장동민의크라우드pc방,D01,3,5.0,5.0,5.0,0,...,2,2,4,2,3,5,1,2인 가족 여행,1,2
31774,2208230010,d_d000621,d000621,에코티엘,D01,24,5.0,5.0,5.0,2,...,2,2,4,2,3,5,1,2인 가족 여행,1,2
31775,2208230007,d_d000621,d000621,전주아줌마두루치기,D01,11,5.0,5.0,5.0,0,...,2,2,4,2,3,5,1,2인 가족 여행,1,2


## 모델 호출

In [6]:
loaded_model = CatBoostRegressor()
loaded_model.load_model('./Model/catboost_model_type_1.cbm')

# 1. 장소 추천

In [7]:
t_features = [
    # 'GENDER', # 성별 - c
    # 'AGE_GRP', # 나이 - c
    # 'INCOME', # 소득
    ## 'VISIT_AREA_TYPE_CD', # 방문지 유형 코드 - c
    # 'TRAVEL_COMPANIONS_NUM', # 동반자수
    'TRAVEL_STATUS_ACCOMPANY', # 동반자 유형
    'TRAVEL_MISSION_PRIORITY', # 개별 미션 최우선 순위 - c
    'TRAVEL_STYL_1', # 자연, 도시
    # 'TRAVEL_STYL_2', # 숙박, 당일
    # 'TRAVEL_STYL_3', # 새로운 지역, 익숙한 지역
    'TRAVEL_STYL_5', # 휴양/휴식, 체험활동
    'TRAVEL_STYL_6', # 유명지
    # 'TRAVEL_STYL_8', # 계획, 상황 에 따른 여행
    # 'RES_CAFE', # 카페 여부 - c
    # 'TRAVEL_MOTIVE_1', # 여행 동기
    # 'VISIT_AREA_NM' # 방문지 이름 - c
    ]

## 전체 장소 리스트 생성(places)

In [8]:
places = df[['VISIT_AREA_NM','VISIT_AREA_TYPE_CD']].drop_duplicates()
places.reset_index(drop = True, inplace = True)
places

,VISIT_AREA_NM,VISIT_AREA_TYPE_CD
0,산굼부리,1
1,한림리조트,24
2,스누피가든,6
3,협재해수욕장,1
4,카페노티드 제주애월,11
...,...,...
8395,곰돌식당,11
8396,미니K코인노래연습장,4
8397,애월해안도로,7
8398,장동민의크라우드pc방,3


In [9]:
name = places['VISIT_AREA_NM'].drop_duplicates()
name.reset_index(drop = True, inplace = True)
name

0              산굼부리
1             한림리조트
2             스누피가든
3            협재해수욕장
4        카페노티드 제주애월
           ...     
7593           곰돌식당
7594     미니K코인노래연습장
7595         애월해안도로
7596    장동민의크라우드pc방
7597      전주아줌마두루치기
Name: VISIT_AREA_NM, Length: 7598, dtype: object

In [10]:
places[places['VISIT_AREA_NM'] == '산굼부리']

,VISIT_AREA_NM,VISIT_AREA_TYPE_CD
0,산굼부리,1
4118,산굼부리,2
4302,산굼부리,7
6303,산굼부리,13


## 예시 데이터 입력(traveler 딕셔너리)

In [11]:
values = ["3인 이상 여행(가족 외)",5, 1, 2, 2]
traveler = dict(zip(t_features, values))
traveler

{'TRAVEL_STATUS_ACCOMPANY': '3인 이상 여행(가족 외)',
 'TRAVEL_MISSION_PRIORITY': 5,
 'TRAVEL_STYL_1': 1,
 'TRAVEL_STYL_5': 2,
 'TRAVEL_STYL_6': 2}

In [12]:
# 데이터프레임에 새로운 컬럼 추가
for col, value in traveler.items():
    places[col] = value

In [13]:
places

,VISIT_AREA_NM,VISIT_AREA_TYPE_CD,TRAVEL_STATUS_ACCOMPANY,TRAVEL_MISSION_PRIORITY,TRAVEL_STYL_1,TRAVEL_STYL_5,TRAVEL_STYL_6
0,산굼부리,1,3인 이상 여행(가족 외),5,1,2,2
1,한림리조트,24,3인 이상 여행(가족 외),5,1,2,2
2,스누피가든,6,3인 이상 여행(가족 외),5,1,2,2
3,협재해수욕장,1,3인 이상 여행(가족 외),5,1,2,2
4,카페노티드 제주애월,11,3인 이상 여행(가족 외),5,1,2,2
...,...,...,...,...,...,...,...
8395,곰돌식당,11,3인 이상 여행(가족 외),5,1,2,2
8396,미니K코인노래연습장,4,3인 이상 여행(가족 외),5,1,2,2
8397,애월해안도로,7,3인 이상 여행(가족 외),5,1,2,2
8398,장동민의크라우드pc방,3,3인 이상 여행(가족 외),5,1,2,2


In [14]:
places.columns

Index(['VISIT_AREA_NM', 'VISIT_AREA_TYPE_CD', 'TRAVEL_STATUS_ACCOMPANY',
       'TRAVEL_MISSION_PRIORITY', 'TRAVEL_STYL_1', 'TRAVEL_STYL_5',
       'TRAVEL_STYL_6'],
      dtype='object')

In [15]:
check = places[['VISIT_AREA_TYPE_CD', 'TRAVEL_STATUS_ACCOMPANY',
       'TRAVEL_MISSION_PRIORITY', 'TRAVEL_STYL_1', 'TRAVEL_STYL_5',
       'TRAVEL_STYL_6']]
check

,VISIT_AREA_TYPE_CD,TRAVEL_STATUS_ACCOMPANY,TRAVEL_MISSION_PRIORITY,TRAVEL_STYL_1,TRAVEL_STYL_5,TRAVEL_STYL_6
0,1,3인 이상 여행(가족 외),5,1,2,2
1,24,3인 이상 여행(가족 외),5,1,2,2
2,6,3인 이상 여행(가족 외),5,1,2,2
3,1,3인 이상 여행(가족 외),5,1,2,2
4,11,3인 이상 여행(가족 외),5,1,2,2
...,...,...,...,...,...,...
8395,11,3인 이상 여행(가족 외),5,1,2,2
8396,4,3인 이상 여행(가족 외),5,1,2,2
8397,7,3인 이상 여행(가족 외),5,1,2,2
8398,3,3인 이상 여행(가족 외),5,1,2,2


In [16]:
loaded_model.predict(check)

array([1.68464108, 1.59385031, 1.62394212, ..., 1.76193375, 1.61193935,
       1.59698894])

In [17]:
places['SCORE'] = loaded_model.predict(check)
places

,VISIT_AREA_NM,VISIT_AREA_TYPE_CD,TRAVEL_STATUS_ACCOMPANY,TRAVEL_MISSION_PRIORITY,TRAVEL_STYL_1,TRAVEL_STYL_5,TRAVEL_STYL_6,SCORE
0,산굼부리,1,3인 이상 여행(가족 외),5,1,2,2,1.684641
1,한림리조트,24,3인 이상 여행(가족 외),5,1,2,2,1.593850
2,스누피가든,6,3인 이상 여행(가족 외),5,1,2,2,1.623942
3,협재해수욕장,1,3인 이상 여행(가족 외),5,1,2,2,1.684641
4,카페노티드 제주애월,11,3인 이상 여행(가족 외),5,1,2,2,1.596989
...,...,...,...,...,...,...,...,...
8395,곰돌식당,11,3인 이상 여행(가족 외),5,1,2,2,1.596989
8396,미니K코인노래연습장,4,3인 이상 여행(가족 외),5,1,2,2,1.529665
8397,애월해안도로,7,3인 이상 여행(가족 외),5,1,2,2,1.761934
8398,장동민의크라우드pc방,3,3인 이상 여행(가족 외),5,1,2,2,1.611939


In [18]:
# SCORE를 기준으로 정렬
recommend_list = places.sort_values('SCORE', ascending=False).reset_index(drop=True)
recommend_list.drop_duplicates()
recommend_list[:10]

,VISIT_AREA_NM,VISIT_AREA_TYPE_CD,TRAVEL_STATUS_ACCOMPANY,TRAVEL_MISSION_PRIORITY,TRAVEL_STYL_1,TRAVEL_STYL_5,TRAVEL_STYL_6,SCORE
0,올레길 14-1코스(저지-서광 올레),7,3인 이상 여행(가족 외),5,1,2,2,1.761934
1,사려니숲길,7,3인 이상 여행(가족 외),5,1,2,2,1.761934
2,김녕바닷길,7,3인 이상 여행(가족 외),5,1,2,2,1.761934
3,제주스위스마을,7,3인 이상 여행(가족 외),5,1,2,2,1.761934
4,성산일출봉,7,3인 이상 여행(가족 외),5,1,2,2,1.761934
5,도두항,7,3인 이상 여행(가족 외),5,1,2,2,1.761934
6,무지개 해안도로,7,3인 이상 여행(가족 외),5,1,2,2,1.761934
7,비자림,7,3인 이상 여행(가족 외),5,1,2,2,1.761934
8,비양도,7,3인 이상 여행(가족 외),5,1,2,2,1.761934
9,제주조천스위스마을,7,3인 이상 여행(가족 외),5,1,2,2,1.761934


In [19]:
recommend_list['VISIT_AREA_NM'][:10]

0    올레길 14-1코스(저지-서광 올레)
1                   사려니숲길
2                   김녕바닷길
3                 제주스위스마을
4                   성산일출봉
5                     도두항
6                무지개 해안도로
7                     비자림
8                     비양도
9               제주조천스위스마을
Name: VISIT_AREA_NM, dtype: object